In [ ]:
import numpy as np
from lstchain.image.muon.muon_analysis import pixel_coords_to_telescope
from ctapipe.io import EventSource
import glob
from ctapipe.image import tailcuts_clean
from ctapipe.visualization import CameraDisplay
import matplotlib.pyplot as plt
from ctapipe.image.muon import (
    MuonIntensityFitter,
    MuonRingFitter
)
import h5py
import sys
sys.path.append('/Users/vdk/Software/code')
from python_modules.muonpipe.usefull_func import get_muon_parameters

filename = '/Users/vdk//muons2024/data/for_comparison/nsb_tuning/triple_tuning/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 1)
tel_description = source.subarray.tels[1]
geom = tel_description.camera.geometry
equivalent_focal_length = tel_description.optics.equivalent_focal_length
x_coord, y_coord = pixel_coords_to_telescope(geom, equivalent_focal_length)

In [ ]:
image_folder = glob.glob('/Users/vdk/muons2024/data/additional_statistic/event_ids/image_17743/image/*')
image_folder = glob.glob('/Users/vdk/muons2024/data/17043/muon_images/*')
data_dl1_images = []

for file in image_folder:
    tmp_result = np.loadtxt(file)
    data_dl1_images.append(tmp_result)

In [ ]:
camgeom = source.subarray.tel[1].camera.geometry
new_data_mask = []
for image in data_dl1_images:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_data_mask.append(mask)

In [ ]:
new_tailcuts_data_mask = []

for image in data_dl1_images:
    negative_Q = np.sort(image[image <= 0])

    hist, bins = np.histogram(negative_Q,range=(-15,0),bins=30)
    bins = bins[:-1]

    cumulative = np.cumsum(hist)
    idx = (np.abs(cumulative - 0.318 * cumulative[-1])).argmin() #Find q closest to standard deviation
    dev = np.abs(bins[idx])

    tailcuts = [4*dev,2*dev]   # tailcuts are placed at 4*dev of each image.
    
    mask = tailcuts_clean(camgeom, image, picture_thresh=tailcuts[0], boundary_thresh=tailcuts[1])
    new_tailcuts_data_mask.append(mask)

In [ ]:
for cur_index in range(50):
    fig, ax = plt.subplots(1, 3, figsize=(26, 10))
    disp = CameraDisplay(
        camgeom, image=data_dl1_images[cur_index], cmap="grey", ax = ax[0], title="DL1 image"
    )
    disp = CameraDisplay(
        camgeom, image=data_dl1_images[cur_index], cmap="grey", ax = ax[1], title="DL1 image with highlighted mask for standard tailcut cleaning [10;5]"
    )
    disp.highlight_pixels(new_data_mask[cur_index], alpha=0.6, linewidth=2, color="red")
    disp = CameraDisplay(
        camgeom, image=data_dl1_images[cur_index], cmap="grey", ax = ax[2], title=f"DL1 image with highlighted mask for modified tailcuts cleaning"
    )
    disp.highlight_pixels(new_tailcuts_data_mask[cur_index], alpha=0.6, linewidth=2, color="red")

In [ ]:
fitter = MuonRingFitter(fit_method='kundu_chaudhuri')

ring = fitter(x_coord, y_coord, data_dl1_images[2], new_data_mask[2])

for _ in (0, 0):  # just to iterate the fit twice more
    dist = np.sqrt(
        (x_coord - ring.center_fov_lon) ** 2 + (y_coord - ring.center_fov_lat) ** 2
    )
    ring_dist = np.abs(dist - ring.radius)

    new_data_mask[2] *= (ring_dist < ring.radius * 3)
    ring = fitter(x_coord, y_coord, image, new_data_mask[2])   

ring.radius

# GlobalPeakWindow processed muon simulations

In [ ]:
# It can be any dl1 file, from real data or simulations
dl1_file = '/Users/vdk/muons2024/data/for_comparison/zenith10/global_peak_window_sum/dl1_run101_muon.h5'

dl1_images = []

with h5py.File(dl1_file, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    
    # Read the data
    data = dataset[:]
    for i, dataline in enumerate(data):
        dl1_images.append(dataline[2])

In [ ]:
new_tailcuts_sim_mask = []

for image in dl1_images:
    negative_Q = np.sort(image[image <= 0])

    hist, bins = np.histogram(negative_Q,range=(-15,0),bins=30)
    bins = bins[:-1]

    cumulative = np.cumsum(hist)
    idx = (np.abs(cumulative - 0.318 * cumulative[-1])).argmin() #Find q closest to standard deviation
    dev = np.abs(bins[idx])

    tailcuts = [4*dev,2*dev]   # tailcuts are placed at 4*dev of each image.
    
    mask = tailcuts_clean(camgeom, image, picture_thresh=tailcuts[0], boundary_thresh=tailcuts[1])
    new_tailcuts_sim_mask.append(mask)

In [ ]:
new_sim_mask = []
for image in dl1_images:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_sim_mask.append(mask)

In [ ]:
for cur_index in range(50):
    fig, ax = plt.subplots(1, 3, figsize=(26, 10))
    
    disp = CameraDisplay(
        camgeom, image=dl1_images[cur_index], cmap="grey", ax = ax[0], title="DL1 image"
    )
    
    disp = CameraDisplay(
        camgeom, image=dl1_images[cur_index], cmap="grey", ax = ax[1], title="DL1 image with highlighted mask for standard tailcut cleaning [10;5]"
    )
    disp.highlight_pixels(new_sim_mask[cur_index], alpha=0.6, linewidth=2, color="red")
    
    disp = CameraDisplay(
        camgeom, image=dl1_images[cur_index], cmap="grey", ax = ax[2], title=f"DL1 image with highlighted mask for modified tailcuts cleaning"
    )
    disp.highlight_pixels(new_tailcuts_sim_mask[cur_index], alpha=0.6, linewidth=2, color="red")

## Real data muon distribution for different tailcut process

In [ ]:
import pandas as pd
from astropy.io import fits

new_tailcuts_dir = '/Users/vdk/muons2024/data/17744/tables_new_tailcuts/*'
standard_tailcuts_dir = '/Users/vdk/muons2024/data/17744/tables_standard_tailcuts/*'
nsbtune2024_reflectivity_dir = '/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity/tables/*'

def process_fits_files(directory):
    fits_files = glob.glob(directory)
    data_frames = []
    
    for file in fits_files:
        with fits.open(file) as hdul:
            data = hdul[1].data
            df = pd.DataFrame(data)
            data_frames.append(df)
    
    return pd.concat(data_frames, ignore_index=True)

new_tailcuts_df = process_fits_files(new_tailcuts_dir)
standard_tailcuts_df = process_fits_files(standard_tailcuts_dir)
sim_2024nsb_reflectivity_df = process_fits_files(nsbtune2024_reflectivity_dir)


In [ ]:
new_tailcuts_df.columns

In [ ]:
standard_tailcuts_df

In [ ]:
# BEGIN: Create histograms for 'ring_completeness' column
plt.figure(figsize=(12, 7))

z,x,c = plt.hist(new_tailcuts_df['ring_completeness'], bins=30, alpha=0.7, label='New tailcuts', density=True)
z,x,c = plt.hist(standard_tailcuts_df['ring_completeness'],bins=30, alpha=0.7, label='Standard tailcuts', density=True)
z,x,c = plt.hist(sim_2024nsb_reflectivity_df['ring_completeness'], bins=30, alpha=0.7, label='2024 NSB tune', density=True)

plt.legend()

In [ ]:
# BEGIN: Create histograms for 'ring_completeness' column
plt.figure(figsize=(12, 7))

z,x,c = plt.hist(new_tailcuts_df['ring_radius'], bins=30, alpha=0.7, label='New tailcuts', density=True)
z,x,c = plt.hist(standard_tailcuts_df['ring_radius'],bins=30, alpha=0.7, label='Standard tailcuts', density=True)
z,x,c = plt.hist(sim_2024nsb_reflectivity_df['ring_radius'], bins=30, alpha=0.7, label='2024 NSB tune', density=True)

plt.legend()

In [ ]:
# BEGIN: Create histograms for 'ring_completeness' column
plt.figure(figsize=(12, 7))

z,x,c = plt.hist(new_tailcuts_df['ring_size'], bins=30, alpha=0.7, label='New tailcuts', density=True)
z,x,c = plt.hist(standard_tailcuts_df['ring_size'],bins=30, alpha=0.7, label='Standard tailcuts', density=True)
z,x,c = plt.hist(sim_2024nsb_reflectivity_df['ring_size'], bins=30, alpha=0.7, label='2024 NSB tune', density=True)

plt.legend()

In [ ]:
# BEGIN: Create histograms for 'ring_completeness' column
plt.figure(figsize=(12, 7))

z,x,c = plt.hist(new_tailcuts_df['muon_efficiency'], bins=300, alpha=0.7, label='New tailcuts', density=True)
z,x,c = plt.hist(standard_tailcuts_df['muon_efficiency'],bins=3000, alpha=0.7, label='Standard tailcuts', density=True)
z,x,c = plt.hist(sim_2024nsb_reflectivity_df['muon_efficiency'], bins=10000, alpha=0.7, label='2024 NSB tune', density=True)

plt.xlim(0,1)
#plt.ylim(0,1)
plt.legend()

In [ ]:
sim_2024nsb_reflectivity_df['muon_efficiency'].mean()

In [ ]:
new_tailcuts_df['muon_efficiency'].mean()   

In [ ]:
standard_tailcuts_df['muon_efficiency'].mean()  

In [ ]:
z,x,c = plt.hist(standard_tailcuts_df['muon_efficiency'],bins=30, alpha=0.7, label='Standard tailcuts', density=True)

In [ ]:
standard_tailcuts_data = get_muon_parameters(glob.glob(standard_tailcuts_dir), muon_efficiency=(1, 'upper'), size_outside=(500, 'upper'))
new_tailcuts_data = get_muon_parameters(glob.glob(new_tailcuts_dir), muon_efficiency=(1, 'upper'), size_outside=(500, 'upper'))9
sim_2024nsb = get_muon_parameters(glob.glob(nsbtune2024_reflectivity_dir), muon_efficiency=(1, 'upper'), size_outside=(500, 'upper'))

In [ ]:
# BEGIN: Create histograms for 'ring_completeness' column
plt.figure(figsize=(12, 7))

z,x,c = plt.hist(new_tailcuts_data['muon_efficiency'], bins=300, alpha=0.7, label='New tailcuts', density=True)
z,x,c = plt.hist(standard_tailcuts_data['muon_efficiency'],bins=100, alpha=0.7, label='Standard tailcuts', density=True)
z,x,c = plt.hist(sim_2024nsb['muon_efficiency'], bins=100, alpha=0.7, label='2024 NSB tune', density=True)

plt.xlim(0,1)
#plt.ylim(0,1)
plt.legend()

In [ ]:
new_tailcuts_data['muon_efficiency'].mean()

In [ ]:
new_tailcuts_df['muon_efficiency'].mean()

In [ ]:
standard_tailcuts_data['muon_efficiency'].mean()

In [ ]:
standard_tailcuts_df['muon_efficiency'].mean()

In [ ]:
max(new_tailcuts_df['muon_efficiency'])

In [ ]:
max(new_tailcuts_data['muon_efficiency'])   

In [ ]:
max(standard_tailcuts_df['muon_efficiency'])    

In [ ]:
max(standard_tailcuts_data['muon_efficiency'])

In [ ]:
max(sim_2024nsb_reflectivity_df['muon_efficiency'])

In [ ]:
max(sim_2024nsb['muon_efficiency']) 